# CEO-REQUEST CHALLENGE

> Should Olist remove underperforming sellers from its marketplace?

## Problem statement

To analyse the impact of removing the worse sellers from the marketplace, we can start with a what-if analysis: what would have happened if Olist had never accepted these sellers in the past? For that:

Step ① Compute, for each `seller_id`, cumulated since the beginning:
- The `revenues` they bring
- The `costs` associated with all their bad reviews
- The resulting `profits` (revenues - costs)
- The number of `orders` (it will impact the overall IT costs)


Step ② We can then sort sellers by increasing profits for Olist, and for different numbers of sellers removed, compute the financial impact it would have made had they never been accepted on the platform. We may find an optimal number of sellers to remove that maximizes Olist's profit margin

In [1]:
# Write down a detailed strategy (step by step) to create the DataFrame you need for step (1)
# Think about how to re-use logic you have already coded in your `olist` package

<details>
    <summary>Hints</summary>


Starting from your current `seller().get_training_data()` DataFrame:
- Can you easily transform it to compute Olist's positive `revenue_per_seller`? 
- Can you easily transform it to compute Olist's `cost_of_bad_reviews`?

❓Instead of starting again from scratch, investigate in your source code in `seller.py` how the mean `review_score` per seller was computed. Can you imagine a way to ammend your code so as to compute `cost_of_reviews` in the same process? 
</details>


## Your turn!

In [2]:
# Keep this notebook tidy, you will present it to the Olist CEO on Friday afternoon!

# Solution (remove from data-challenge)

Suggested strategy:
- Update `seller.py` to compute a missing `cost_of_reviews` column
- Compute `months_on_olist` and `revenues` using existing seller_training_data columns
- Compute `profits` per seller
- Sort DataFrame per `profits`
- Remove worst performing sellers one by one, and for each scenario:
    - compute `n_orders` and therefore `olist_it_costs` (proportional to square root of n_orders)
    - compute Olist's total `net_profits` and `margin`
- Find the optimal number of sellers to remove from the platform so as to maximize `profit` or `margin`

## Step ① : Update `seller.py`

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

In [4]:
# seller.py has been updated to seller_updated.py to automatically compute what we need for step (1)
from olist.seller_updated import Seller
seller = Seller()

In [5]:
sellers = seller.get_training_data()
sellers

## Step ② : What-if analysis

### Olist P&L (big picture)

_Revenues_

In [6]:
# Gross Revenues
olist_revenues = sellers.revenues.sum()
olist_revenues

In [7]:
# Including from monthly subscription
olist_subscription_revenues = sellers.months_on_olist.sum()*80
olist_subscription_revenues

In [8]:
# Including from sales
olist_sales_revenues = sellers.sales.sum()*0.1
olist_sales_revenues

_Costs_

In [9]:
olist_reputation_costs = sellers.cost_of_reviews.sum()
olist_reputation_costs

In [10]:
olist_it_costs_all_orders = 500000

In [11]:
# Total costs
olist_total_costs = 500000 + sellers.cost_of_reviews.sum()
olist_total_costs

_Profits_

In [12]:
# before IT costs
olist_gross_profits = sellers.profits.sum()
olist_gross_profits

In [13]:
# after IT costs
olist_net_profits = olist_gross_profits - olist_it_costs_all_orders
olist_net_profits

In [14]:
import plotly.graph_objects as go

fig = go.Figure(go.Waterfall(
    orientation = "v",
    measure = ["relative", "relative", "total", "relative", "relative", "total"],
    x = ["Monthly subcriptions", "Sales", "Total Revenues", "Reputation costs", "It costs", "Net Profit"],
    textposition = "outside",
#    text = ["", "+80", "", "-40", "-20", "Total"],
    y = [olist_subscription_revenues, olist_sales_revenues, 0, -olist_reputation_costs, -olist_it_costs_all_orders, 0],
    connector = {"line":{"color":"rgb(63, 63, 63)"}},
))

fig.update_layout(
        title = "Olist P&L since inception (Million BRL)",
        showlegend = True
)

fig.show()

### Seller cut-off analysis

In [15]:
sns.histplot(sellers.profits, kde=False)

In [16]:
sorted_sellers = sellers.sort_values(by='profits')[['profits', 'n_orders', 'revenues']].reset_index()
sorted_sellers.head()

In [17]:
sorted_sellers.profits.plot();

#### Compute Olist's net_profits for various seller cut-offs

Let's create arrays `xxx_per_seller_removed` which list the `xxx` value for each number of sellers removed (from 1 to 3000 sellers)


In [18]:
# Analysis excluding IT costs
revenues_per_seller_removed = olist_revenues - np.cumsum(sorted_sellers.revenues)
gross_profits_per_seller_removed = olist_gross_profits - np.cumsum(sorted_sellers.profits)

gross_profits_per_seller_removed.plot()
plt.title('Olist gross profit before IT costs')
plt.xlabel('number of seller to remove')

In [19]:
# Add the IT costs of Olist's platform
# IT costs =  A * (n_orders)**0.5

# First, compute A
A = olist_it_costs_all_orders / (sellers['n_orders'].sum()**0.5)
A

In [20]:
# Then, compute n_order per scenario
n_orders_per_seller_removed = sorted_sellers.n_orders.sum() - np.cumsum(sorted_sellers.n_orders)
plt.plot(n_orders_per_seller_removed)
plt.title('Total number of orders'); plt.xlabel('number of seller to remove');

In [21]:
# Create array of it_costs per scenario
it_costs_per_seller_removed = A * n_orders_per_seller_removed**0.5
plt.plot(it_costs_per_seller_removed)
plt.title('IT costs'); plt.xlabel('number of seller to remove');

In [22]:
# Compute final metrics (profits and margin, for each scenario!)
profits_per_seller_removed = gross_profits_per_seller_removed - it_costs_per_seller_removed
margin_per_seller_removed = profits_per_seller_removed / revenues_per_seller_removed

In [23]:
fig, ax1 = plt.subplots(figsize=(10,5))
x = np.arange(0, len(sellers), 1)

plt.suptitle("Estimated impact of having excluding selected sellers from Olist, over the past 16 months")

ax1.set_xlabel('number of seller to remove from olist - from worse to most profitable')
ax1.set_ylabel('BRL')
ax1.plot(x, profits_per_seller_removed, color='black', label='Olist net profits')
ax1.plot(x, revenues_per_seller_removed, color='grey', label='Olist total revenues')
ax1.legend()
ax1.tick_params(axis='y')
ax1.legend()
ax1.set(ylim=[0,3000000])


ax3 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax3.set_ylabel('%', color=color)  # we already handled the x-label with ax1
ax3.plot(x, margin_per_seller_removed, color='red', label='Olist profit margin')
ax3.tick_params(axis='y', labelcolor=color)
ax3.legend(loc='right')
ax3.set(ylim=[0,1])

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

☝️We can clearly see a trade-off with a maximim net_profit at around 500 sellers removed, and a maximum margin at about 2000 sellers removed. Beyond this, the margin collapses due to IT costs.

🔎 Going further, we should improve this analysis by taking into account the dynamics: Olist does not know in advance a seller's performance. Sellers should only be removed from the platform after they have been receiving bad reviews for a given period.